<a href="https://colab.research.google.com/github/blinders79/product-search/blob/main/ProductSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install --upgrade openai httpx
!pip install --upgrade openai
!pip install httpx==0.23.0 # Downgrade httpx to a compatible version
!pip install python-dotenv

import pandas as pd
import numpy as np
import os
from openai import OpenAI
from dotenv import load_dotenv
from sklearn.metrics.pairwise import cosine_similarity


def get_recommendations(inputText):
  # Loading API key and organization ID from a dotenv file
  load_dotenv(dotenv_path='apikey.env.txt')
  # Retrieving API key and organization ID from environment variables
  APIKEY = os.getenv("APIKEY")
  # Set your API key
  client = OpenAI(api_key=APIKEY)

  # Make a request to the GPT-4 model
  #The openai.ChatCompletion.create method has been replaced with client.chat.completions.create
  # in the latest version of the openai library.
  products_data = pd.read_csv("/content/USA_Banking_Products.csv")
  products_data["item"] = products_data['Product Type'] + products_data['Account Type'] + products_data['Benefit']
  response = client.embeddings.create(
      input=products_data.item.to_list(),
      model="text-embedding-3-small",
      dimensions=512
  )
  vector_embeddings = [d.embedding for d in response.data]
  products_data['embedding_vector'] = vector_embeddings
  user_prompt = inputText
  response = client.embeddings.create(
      input=[user_prompt],
      model="text-embedding-3-small",
      dimensions=512
  )
  user_prompt_vector = [d.embedding for d in response.data]
  product_vectors = [np.array(vec) for vec in products_data['embedding_vector']]
  user_prompt_vector = np.array(user_prompt_vector)
  similarity_matrix = cosine_similarity(product_vectors,user_prompt_vector)
  similarity_values = [v[0] for v in similarity_matrix]
  products_data['similarity_values'] = similarity_values
  products_data.sort_values(by='similarity_values', ascending=False, inplace=True)
  return products_data[['Bank Name','Product Type','Account Type','Benefit']].head(2).to_dict(orient="records")


def generate_chatgpt_response(user_input):
    # Loading API key and organization ID from a dotenv file
    load_dotenv(dotenv_path='apikey.env.txt')
    # Retrieving API key and organization ID from environment variables
    APIKEY = os.getenv("APIKEY")
    # Set your API key
    client = OpenAI(api_key=APIKEY)
    top_responses = get_recommendations(user_input)
    #print(top_responses)

    prompt = f"User: {user_input}\n"
    prompt += f"Assistant (based **only** on the following matches):\n1. {top_responses[0]}\n2. {top_responses[1]}\n\n"
    prompt += "Generate a response using **only** the information provided above. Do not include any external knowledge or assumptions."  # Emphasize restriction

    completion = client.chat.completions.create(
        model="gpt-4",
        messages=[
            {"role": "system", "content": "You are a helpful AI assistant that provides responses based solely on provided information."},  # Set system role
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": f"Here are the most relevant matches:\n1. {top_responses[0]}\n2. {top_responses[1]}"}
        ]
    )

    #print(completion.choices[0])

    # Try this first
    # return completion.choices[0].message.content

    # If above line fails, try this
    return completion.choices[0].message.content

print(generate_chatgpt_response("Top Luxury Cards"))



  Using cached httpx-0.28.1-py3-none-any.whl.metadata (7.1 kB)
  Using cached httpcore-1.0.7-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
Using cached httpx-0.28.1-py3-none-any.whl (73 kB)
Using cached httpcore-1.0.7-py3-none-any.whl (78 kB)
Using cached h11-0.14.0-py3-none-any.whl (58 kB)
  Attempting uninstall: h11
    Found existing installation: h11 0.12.0
    Uninstalling h11-0.12.0:
      Successfully uninstalled h11-0.12.0
  Attempting uninstall: httpcore
    Found existing installation: httpcore 0.15.0
    Uninstalling httpcore-0.15.0:
      Successfully uninstalled httpcore-0.15.0
  Attempting uninstall: httpx
    Found existing installation: httpx 0.23.0
    Uninstalling httpx-0.23.0:
      Successfully uninstalled httpx-0.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-genai 1.7.0 requires

In [2]:
!pip install flask flask-ngrok
!pip install fastapi uvicorn nest-asyncio
!pip install flask pyngrok
!ngrok authtoken "2uoW1dUut1ruqCm63wSUXxukjIP_3676oEoZefzqpEgqTW5AL"

from flask import Flask, request, jsonify
from pyngrok import ngrok

# Create Flask app
app = Flask(__name__)

@app.route("/")
def home():
    return "API is running!"

@app.route("/products", methods=["POST"])
def run_code():
    data = request.get_json()
    print(data)
    input_text = data.get("prompt", "Top Luxury Cards")
    responseData = generate_chatgpt_response(input_text)
    #response_data = df.to_dict(orient="records")
    return responseData

# Start the Flask app
public_url = ngrok.connect(5000).public_url
print(f"Public URL: {public_url}")

app.run(port=5000)

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml
Public URL: https://08b4-34-85-175-37.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit


{'prompt': 'get me best credit card for travel'}


INFO:werkzeug:127.0.0.1 - - [26/Mar/2025 09:18:54] "POST /products HTTP/1.1" 200 -


,Bank Name,Card Name,Category,Key Benefits
0,Citibank,Citi Rewards Credit Card,Shopping,Earn reward points on apparel and department s...
1,Citibank,Citi PremierMiles Credit Card,Travel,"Earn air miles on all purchases, redeemable fo..."
2,Citibank,Citi Cash Back Credit Card,Cashback,"Get cashback on utility bill payments, movie t..."
3,Bank of America,Customized Cash Rewards Credit Card,Cashback,Earn 3% cash back in a category of your choice.
4,Bank of America,Travel Rewards Credit Card,Travel,Earn unlimited 1.5 points per $1 spent on all ...
5,Bank of America,Premium Rewards Credit Card,Premium,"Earn 2X points on travel and dining, 1.5X poin..."
6,American Express,The Platinum Card,Premium Travel,Luxury benefits including airport lounge acces...
7,American Express,Gold Card,Dining & Travel,Earn 4X points at restaurants and supermarkets.
8,American Express,Blue Cash Preferred,Cashback,Earn 6% cash back on U.S. supermarkets and str...
9,Chase,Chase Sapphire Preferred,Travel,"Earn 2X points on travel and dining, premium t..."


In [ ]:

print(user_prompt_vector[:10])
#

[[0.03022068366408348, -0.06338129192590714, 0.02176840417087078, -0.03525746613740921, -0.04232625290751457, -0.02676195092499256, -0.033701036125421524, 0.1240820661187172, 0.05719880759716034, -0.010916627943515778, 0.010181646794080734, -0.011478671804070473, -0.08011291921138763, 0.03642478585243225, 0.04600115492939949, -0.04262889176607132, -0.06830998510122299, 0.03212298825383186, 0.033787503838539124, 0.03692198172211647, 0.006236528977751732, 0.0903594121336937, -0.003291201079264283, -0.03951603174209595, 0.014126764610409737, 0.02568109706044197, -0.0005400893278419971, -0.03344162926077843, 0.05845259502530098, -0.056593526154756546, 0.0734548568725586, -0.045222941786050797, 0.07946440577507019, -0.011111181229352951, -0.06541329622268677, -0.018385330215096474, 0.015596726909279823, 0.08581982553005219, 0.07224429398775101, -0.03560333698987961, -0.03039362095296383, 0.0448770672082901, 0.07021228969097137, 0.005798783153295517, 0.03519261255860329, 0.043688129633665085

,Bank Name,Card Name,Category,Key Benefits,item,embedding_vector,similarity_values
6,American Express,The Platinum Card,Premium Travel,Luxury benefits including airport lounge acces...,Premium TravelLuxury benefits including airpor...,"[0.018366245552897453, -0.02336861751973629, 0...",0.637523
9,Chase,Chase Sapphire Preferred,Travel,"Earn 2X points on travel and dining, premium t...","TravelEarn 2X points on travel and dining, pre...","[-0.027171188965439796, -0.019147811457514763,...",0.507441
11,Chase,Ink Business Preferred Credit Card,Business,Earn 3X points on travel and select business c...,BusinessEarn 3X points on travel and select bu...,"[-0.06839323043823242, 0.01712595857679844, 0....",0.459276
